# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [1]:
import pandas as pd
import sqlite3 
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [2]:
cur.execute("""
    SELECT lastName, firstName 
    FROM employees
    JOIN offices
    USING(officeCode)
    WHERE city = 'Boston';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [3]:
cur.execute("""
    SELECT officeCode, city, COUNT(*) as num_employees
    FROM offices
    LEFT JOIN employees
    USING(officeCode)
    GROUP BY officeCode;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,num_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Write 3 Questions of your own and answer them

In [4]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

### Q1: Which product is ordered the most?

In [5]:
cur.execute("""
    SELECT productCode, productName, COUNT(orderNumber) as num_orders
    FROM products
    JOIN orderDetails
    USING(productCode)
    GROUP BY productCode
    ORDER BY num_orders DESC
    LIMIT 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productCode,productName,num_orders
0,S18_3232,1992 Ferrari 360 Spider red,53


### Q2: Which product was ordered the most in a single order? Identify the orderNumber.

In [6]:
cur.execute("""
    SELECT productCode, productName, MAX(quantityOrdered) as max_single_order, orderNumber
    FROM products
    JOIN orderDetails
    USING(productCode)
    GROUP BY productCode
    ORDER BY max_single_order DESC
    LIMIT 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productCode,productName,max_single_order,orderNumber
0,S12_4675,1969 Dodge Charger,97,10405


### Q3: Are there any products that have not been ordered? If so, identify them by productCode.

In [7]:
cur.execute("""
    SELECT productCode, productName, COUNT(orderNumber)=0 as has_not_been_ordered
    FROM orderDetails
    LEFT JOIN products
    USING(productCode)
    GROUP BY productCode
    HAVING has_not_been_ordered = 1;""")
df = pd.DataFrame(cur.fetchall())
if len(df)>0:
    df.columns = [i[0] for i in cur.description]
else:
    df = pd.DataFrame(columns=[i[0] for i in cur.description])
df

,productCode,productName,has_not_been_ordered


## Level Up: Display the names of every individual product that each employee has sold

In [8]:
cur.execute("""
    SELECT productCode, productName, COUNT(orderNumber) as num_orders
    FROM products
    JOIN orderDetails
    USING(productCode)
    GROUP BY productCode
    ORDER BY num_orders DESC
    LIMIT 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productCode,productName,num_orders
0,S18_3232,1992 Ferrari 360 Spider red,53


## Level Up: Display the Number of Products each employee has sold

In [9]:
cur.execute("""
    SELECT employeeNumber, lastName, firstName, SUM(quantityOrdered) as num_products_sold
    FROM employees
    JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
    JOIN orders
    ON customers.customerNumber = orders.customerNumber
    JOIN orderDetails
    ON orders.orderNumber = orderdetails.orderNumber
    GROUP BY employeeNumber
    ORDER BY num_products_sold DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,num_products_sold
0,1370,Hernandez,Gerard,14231
1,1165,Jennings,Leslie,11854
2,1401,Castillo,Pamela,9290
3,1501,Bott,Larry,8205
4,1504,Jones,Barry,7486
5,1323,Vanauf,George,7423
6,1612,Marsh,Peter,6632
7,1611,Fixter,Andy,6246
8,1337,Bondur,Loui,6186
9,1216,Patterson,Steve,5561


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!